In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import json, os
from collections import Counter, defaultdict, deque

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import (
    classification_report, roc_auc_score, average_precision_score,
    precision_recall_curve, roc_curve, brier_score_loss,
    confusion_matrix, ConfusionMatrixDisplay, make_scorer
)
from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from snowflake.ml.registry import Registry

In [ ]:
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
pd.set_option('display.max_columns', None)

In [ ]:
xf = session.table("HACKATHON.DATAMART.FACT_ML_FEATURES").to_pandas()

In [ ]:
df = xf.copy()

In [ ]:
df.columns = df.columns.str.lower()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.sort_values('timestamp_1h').reset_index(drop=True)

In [ ]:
# Plot after sorting
plt.figure(figsize=(10, 4))
plt.plot(df['timestamp_1h'])
plt.title('After Sorting')
plt.xlabel('Index')
plt.ylabel('timestamp_1h')
plt.show()

In [ ]:
# Sort by the column 'ROW_NUMBER' and reset the index
df = df.sort_values(by='row_number').reset_index(drop=True)

In [ ]:
df.head(2)

In [ ]:
df.tail(2)

In [ ]:
# Dataset split:
# - 20% for test
# - 10% of the remaining data for validation
# - 5% of the training data for calibration (make probability predictions more accurate)
test_frac = 0.20
val_frac  = 0.10
cal_frac  = 0.05

n = len(df)
test_cut = int(n * (1 - test_frac))
pre_test = df.iloc[:test_cut].copy()
test_df  = df.iloc[test_cut:].copy()

val_cut  = int(len(pre_test) * (1 - val_frac))
train_df = pre_test.iloc[:val_cut].copy()
val_df   = pre_test.iloc[val_cut:].copy()

cal_cut  = int(len(train_df) * (1 - cal_frac))
train_core = train_df.iloc[:cal_cut].copy()
cal_df     = train_df.iloc[cal_cut:].copy()

for name, df_ in [("train_core", train_core), ("cal", cal_df), ("val", val_df), ("test", test_df)]:
    print(name, df_['timestamp_1h'].min(), "→", df_['timestamp_1h'].max(), "rows:", len(df_))

In [ ]:
def show_target(df, name):
    p = df['label_outage_1h'].mean()*100
    print(f"{name:<10} pos%={p:.4f}  (pos={df['label_outage_1h'].sum():,}, n={len(df):,})")

show_target(train_core, "train_core")
show_target(cal_df,     "cal")
show_target(val_df,     "val")
show_target(test_df,    "test")

In [ ]:
# Numeric non-binary
selected_feats = [
    'ont_registered',
    'offline_ont_now',
    'offline_ont_ratio',
    'link_loss_count',
    'bad_rsl_count',
    'high_temp_count',
    'dying_gasp_count',
    'trap_trend_score',
    'fault_rate',
    'snr_avg',
    'rx_power_avg',
    'rx_power_avg_dbm',
    'temperature_avg_c',
    'temp_anomaly_score',
    'hour_of_day',
    'day_of_week'
]

# Initialize all with None
capping_limits = {feature: None for feature in selected_feats}

capping_limits['ont_registered'] = [0, 100]
capping_limits['offline_ont_now'] = [0, 100] # extreme outliers, count
capping_limits['offline_ont_ratio'] = [0, 98] # extreme outliers
capping_limits['link_loss_count'] = [0, 100] # extreme outliers, count
capping_limits['bad_rsl_count'] = [0, 100] # extreme outliers, count
capping_limits['high_temp_count'] = [0, 100] # extreme outliers, count
capping_limits['dying_gasp_count'] = [0, 100] # extreme outliers, count
capping_limits['trap_trend_score'] = [1, 98] # extreme outliers
capping_limits['fault_rate'] = [0, 99.5] # extreme outliers
capping_limits['snr_avg'] = [0.5, 100] # extreme outliers
capping_limits['rx_power_avg'] = [0.15, 100] # extreme outliers
capping_limits['rx_power_avg_dbm'] = [0.15, 100] # extreme outliers
capping_limits['temperature_avg_c'] = [0.5, 99.5] # extreme outliers
capping_limits['temp_anomaly_score'] = [0.5, 99.5] # extreme outliers
capping_limits['hour_of_day'] = [0, 100]
capping_limits['day_of_week'] = [0, 100]

def cap_values(series, lower, upper):
    return np.clip(series, lower, upper)

caps = {}

for col, (low_p, high_p) in capping_limits.items():
    # Compute caps from TRAINING data only
    lower_cap = -np.inf if low_p == 0 else np.percentile(train_core[col], low_p)
    upper_cap = np.inf if high_p == 100 else np.percentile(train_core[col], high_p)
    caps[col] = (lower_cap, upper_cap)

    before_min, before_max = train_core[col].min(), train_core[col].max()
    num_low = (train_core[col] < lower_cap).sum()
    num_high = (train_core[col] > upper_cap).sum()

    print(f"\n[{col}]")
    print(f"  Before: min={before_min:.4f}, max={before_max:.4f}")
    print(f"  Caps: lower={lower_cap:.4f}, upper={upper_cap:.4f}")
    print(f"  Rows capped: low={num_low}, high={num_high}")

    for df in [train_core, cal_df, val_df, test_df]:
        df[col] = cap_values(df[col], lower_cap, upper_cap)

    after_min, after_max = train_core[col].min(), train_core[col].max()
    print(f"  After:  min={after_min:.4f}, max={after_max:.4f}")

In [ ]:
# The skewed count-like features
skewed_feats = ['offline_ont_now', 'bad_rsl_count', 'link_loss_count', 'dying_gasp_count', 'high_temp_count']

# Apply log1p transform safely (no drop)
for df in [train_core, cal_df, val_df, test_df]:
    for col in skewed_feats:
        # Add small epsilon to avoid all zeros
        clipped = np.clip(df[col], 0, None)
        df[f'{col}_log'] = np.log1p(clipped + 1e-10)  # Small epsilon

# Build list of log-transformed feature names
skewed_feats_log = [f'{col}_log' for col in skewed_feats]

# Compute skew only for non-binary numeric columns (selected + log-transformed)
skewed = train_core[selected_feats + skewed_feats_log].skew().sort_values(ascending=True)

# Filter skewed Series to exclude features in skewed_feats
skewed_filtered = skewed[~skewed.index.isin(skewed_feats)]

# Print like a pandas Series
for feature, skew_value in skewed_filtered.items():
    print(f"{feature:<20} {skew_value:.3f}")

In [ ]:
# Drop them manually from each dataframe
for df in [train_core, cal_df, val_df, test_df]:
    df.drop(columns=skewed_feats, inplace=True)

In [ ]:
def target_dist(df, col_target, target_1_label='Positive', target_0_label='Negative'):
    # Smaller global font size
    mpl.rcParams['font.size'] = 8

    # Count values
    r = df[col_target].value_counts().sort_index()
    total = r.sum()

    # Labels in correct order (0 → Negative, 1 → Positive)
    labels = [target_0_label if i == 0 else target_1_label for i in r.index]
    
    # Custom function for % and count
    def make_autopct(values):
        def my_autopct(pct):
            val = int(round(pct * total / 100.0))
            return f"{pct:.1f}%\n({val:,})"
        return my_autopct

    # Smaller figure and radius
    fig, ax = plt.subplots(figsize=(2.2, 2.2))  # compact figure
    wedges, texts, autotexts = ax.pie(
        r,
        explode=[0.02, 0.04],
        labels=labels,
        radius=0.9,
        autopct=make_autopct(r),
        shadow=False,
        startangle=45,
        colors=['#66b3ff', '#ff9999'],
        textprops={'color': 'black', 'fontsize': 7}
    )

    # Clean styling
    ax.set_aspect('equal')
    ax.set_frame_on(False)
    plt.setp(autotexts, size=6, weight="bold", color="white")

    # Compact title and layout
    plt.title(f"{col_target} Distribution", fontweight="bold", fontsize=9, pad=6)
    plt.tight_layout(pad=0.5)
    plt.show()

# Example usage
target_dist(train_core, 'label_outage_1h', target_1_label='Outage', target_0_label='No Outage')

In [ ]:
GROUP_KEY = 'olt_id'
ROLL_KEYS = [
    'link_loss_count_log','bad_rsl_count_log','high_temp_count_log',
    'dying_gasp_count_log','offline_ont_now_log'
]

def _safe_numeric(df, cols):
    f = df.copy()
    for c in cols:
        if c in f.columns:
            f[c] = pd.to_numeric(f[c], errors='coerce')
    # Replace inf/-inf from logs, then fill NaN later
    f.replace([np.inf, -np.inf], np.nan, inplace=True)
    return f

def add_time_feats(f):
    f = f.copy()
    f['hour_sin'] = np.sin(2*np.pi * f['hour_of_day']/24.0)
    f['hour_cos'] = np.cos(2*np.pi * f['hour_of_day']/24.0)
    return f

def add_roll_delta(f, group_key=GROUP_KEY, windows=(6, 24)):
    f = _safe_numeric(f, ROLL_KEYS)
    f = f.sort_values(['timestamp_1h', group_key]).copy()
    # Deltas on LOG-space (interpretable as multiplicative change)
    for col in ROLL_KEYS:
        if col in f.columns:
            f[col + "_delta_1h"] = f.groupby(group_key)[col].diff().astype(float)
    # Rolling means on LOG-space (smooths multiplicative noise)
    for w in windows:
        for col in ROLL_KEYS:
            if col in f.columns:
                f[f"{col}_roll{w}h_mean"] = (
                    f.groupby(group_key)[col]
                     .rolling(w, min_periods=1).mean()
                     .reset_index(level=0, drop=True)
                     .astype(float)
                )
    return f

def prepare_block(df):
    f = add_time_feats(df)
    # ensure *_log present; if not, warn minimally
    missing_logs = [c for c in ROLL_KEYS if c not in f.columns]
    if missing_logs:
        print("WARNING: missing log columns:", missing_logs)
    return f

In [ ]:
# Prepare blocks
train_core_f = prepare_block(train_core)
cal_f        = prepare_block(cal_df)
val_f        = prepare_block(val_df)
test_f       = prepare_block(test_df)

# Compute roll/delta chronologically across all to avoid leakage
concat_all = pd.concat([train_core_f, cal_f, val_f, test_f], axis=0).sort_values('timestamp_1h')
concat_all = add_roll_delta(concat_all, group_key=GROUP_KEY, windows=(6,24))

train_core_f = concat_all.loc[train_core_f.index].copy()
cal_f        = concat_all.loc[cal_f.index].copy()
val_f        = concat_all.loc[val_f.index].copy()
test_f       = concat_all.loc[test_f.index].copy()

In [ ]:
# Base numeric (no raw IDs)
num_base = [
    'offline_ont_ratio','trap_trend_score','fault_rate',
    'snr_avg','rx_power_avg_dbm','temperature_avg_c','temp_anomaly_score',
    'hour_sin','hour_cos','is_maintenance_window'
]

# Build feature list: *_log deltas/rollings + base
roll_cols = [c for c in train_core_f.columns if any(s in c for s in [
    "_delta_1h","_roll6h_mean","_roll24h_mean"
]) and any(c.startswith(k) for k in ROLL_KEYS)]

feature_cols_all = [c for c in num_base if c in train_core_f.columns] + roll_cols

# Clean NaNs from log ops later during matrix build
target_col = 'label_outage_1h'
print("Num features:", len(feature_cols_all))
print(sorted(feature_cols_all)[:12], "...")

In [ ]:
col_to_drop = ['temp_anomaly_score']
# Drop the column in-place for each DataFrame
for df in [train_core_f, cal_f, val_f, test_f]:
    df.drop(columns=col_to_drop, inplace=True)

In [ ]:
feature_cols_all = [col for col in feature_cols_all if col not in col_to_drop]

In [ ]:
def make_xy(frame, feats, target):
    X = frame[feats].copy()
    # Replace NaNs introduced by logs/diffs/rolling
    X = X.replace([np.inf, -np.inf], np.nan).fillna(0.0).astype(float)
    y = frame[target].astype(int)
    return X, y

X_tr, y_tr = make_xy(train_core_f, feature_cols_all, target_col)
X_cal, y_cal = make_xy(cal_f, feature_cols_all, target_col)
X_va,  y_va  = make_xy(val_f,  feature_cols_all, target_col)
X_te,  y_te  = make_xy(test_f, feature_cols_all, target_col)

print("Shapes:", X_tr.shape, X_cal.shape, X_va.shape, X_te.shape)

In [ ]:
# Convert feature datasets
X_tr = X_tr.astype('float32')
X_cal = X_cal.astype('float32')
X_va = X_va.astype('float32')
X_te = X_te.astype('float32')

# Convert label datasets
y_tr = y_tr.astype('int8')
y_cal = y_cal.astype('int8')
y_va = y_va.astype('int8')
y_te = y_te.astype('int8')

In [ ]:
def threshold_by_recall(y_true, y_proba, target_recall=0.30, min_precision=0.30):
    """
    Choose threshold that reaches >= target_recall on validation.
    Prefer the candidate with the highest precision; if none reach target,
    pick the max-recall point (best effort).
    """
    p, r, thr = precision_recall_curve(y_true, y_proba)
    p, r = p[1:], r[1:]  # align with thresholds
    idx = np.where(r >= target_recall)[0]
    if len(idx) == 0:
        best = int(np.argmax(r))
        return float(thr[best]), float(p[best]), float(r[best])
    ok = idx[p[idx] >= min_precision]
    if len(ok) == 0:
        ok = idx
    best = ok[np.argmax(p[ok])]
    return float(thr[best]), float(p[best]), float(r[best])

def scan_threshold_grid(y_true, y_proba, recall_targets, precision_floors):
    """
    Grid-search over (target_recall, min_precision) pairs on validation.
    Returns a DataFrame with the chosen threshold and the achieved (precision, recall)
    for each pair, plus a composite score (F_beta) you can use to select the 'best'.
    """
    rows = []
    for tr in recall_targets:
        for mp in precision_floors:
            thr, p_va, r_va = threshold_by_recall(y_true, y_proba, tr, mp)
            beta = 2.0  # emphasize recall (F2)
            if (p_va + r_va) == 0:
                fbeta = 0.0
            else:
                fbeta = (1+beta**2) * (p_va*r_va) / (beta**2*p_va + r_va)
            rows.append({
                "target_recall": tr,
                "min_precision": mp,
                "threshold": thr,
                "precision_val": p_va,
                "recall_val": r_va,
                "f2_val": fbeta
            })
    return pd.DataFrame(rows).sort_values(["f2_val","recall_val","precision_val"], ascending=False)

def pick_best_by_recall(models_probas, y_true, recall_target=0.30, min_precision=0.30):
    """
    models_probas: dict name -> (proba_va, proba_te)
    Returns: dict of metrics keyed by model name using recall-first thresholding
    """
    out = {}
    for name, (proba_va, proba_te) in models_probas.items():
        thr, p_va, r_va = threshold_by_recall(y_true, proba_va, recall_target, min_precision)
        out[name] = (thr, p_va, r_va, proba_te)
    return out

In [ ]:
models = {} # will fill with: name -> (calibrated_model, X_va_matrix, X_te_matrix)
ap_scorer = make_scorer(average_precision_score, needs_proba=True)

# --- subsample 10% for tuning ---
sample_frac = 0.10
X_tune = X_tr.sample(frac=sample_frac, random_state=42)
y_tune = y_tr.loc[X_tune.index]
print(f"Tuning on {len(X_tune):,} rows ({sample_frac*100:.0f}%) of training data")

pos_weight = (len(y_tr) - y_tr.sum()) / y_tr.sum()
print(f"Class imbalance ratio ≈ {pos_weight:.1f}:1")

In [ ]:
xgb = XGBClassifier(
    tree_method="hist", n_jobs=-1, eval_metric="aucpr",
    random_state=RANDOM_STATE
)
grid_xgb = {
    "n_estimators": [400],
    "learning_rate": [0.05],
    "max_depth": [6, 8],
    "subsample": [0.8],
    "colsample_bytree": [0.8],
    "min_child_weight": [1, 3],
    "scale_pos_weight": [pos_weight]
}
gs_xgb = GridSearchCV(xgb, grid_xgb, scoring=ap_scorer, cv=2, n_jobs=1, verbose=1)
gs_xgb.fit(X_tune, y_tune)
best_xgb = gs_xgb.best_estimator_
best_xgb.fit(X_tr, y_tr)
cal_xgb = CalibratedClassifierCV(best_xgb, cv="prefit", method="sigmoid")
cal_xgb.fit(X_cal, y_cal)
models["XGBoost"] = (cal_xgb, X_va, X_te)
print("XGBoost best:", gs_xgb.best_params_)

In [ ]:
cat = CatBoostClassifier(loss_function="Logloss", eval_metric="AUC",
                         random_seed=RANDOM_STATE, verbose=False, thread_count=1)
grid_cat = {
    "iterations": [1200],
    "learning_rate": [0.05],
    "depth": [6, 8],
    "l2_leaf_reg": [3.0],
    "class_weights": [[1.0, pos_weight]]
}
gs_cat = GridSearchCV(cat, grid_cat, scoring=ap_scorer, cv=2, n_jobs=1, verbose=1)
gs_cat.fit(X_tune, y_tune)
best_cat = gs_cat.best_estimator_
best_cat.fit(X_tr, y_tr)
cal_cat = CalibratedClassifierCV(best_cat, cv="prefit", method="sigmoid")
cal_cat.fit(X_cal, y_cal)
models["CatBoost"] = (cal_cat, X_va, X_te)
print("CatBoost best:", gs_cat.best_params_)

In [ ]:
# Cobmpute and cache probabilities for every model
probas = {}  # name -> dict with proba_va, proba_te, references to X sets
for name, (model, Xv, Xt) in models.items():
    proba_va = model.predict_proba(Xv)[:, 1]
    proba_te = model.predict_proba(Xt)[:, 1]
    probas[name] = {"proba_va": proba_va, "proba_te": proba_te, "Xv": Xv, "Xt": Xt}

In [ ]:
# Config
recall_grid = [0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90]
precision_grid = [0.20, 0.30, 0.40, 0.50, 0.60, 0.70]
max_diff = 0.05  # allowed difference between val and test metrics

summary_rows = []
chosen_ops = {}

for name, d in probas.items():
    df_scan = scan_threshold_grid(y_va, d["proba_va"], recall_grid, precision_grid)
    # Sort by F2 (best first), tie-break by recall, then precision
    df_scan = df_scan.sort_values(["f2_val", "recall_val", "precision_val"], ascending=[False, False, False]).reset_index(drop=True)

    found_valid = False

    # Try each candidate threshold in order
    for _, row in df_scan.iterrows():
        thr = float(row["threshold"])
        y_pred_te = (d["proba_te"] >= thr).astype(int)
        tp = int(((y_te == 1) & (y_pred_te == 1)).sum())
        fp = int(((y_te == 0) & (y_pred_te == 1)).sum())
        prec_test = tp / max(tp + fp, 1)
        rec_test  = tp / max(int(y_te.sum()), 1)

        diff_prec = abs(float(row["precision_val"]) - prec_test)
        diff_rec  = abs(float(row["recall_val"]) - rec_test)
        overfit_flag = (diff_prec > max_diff) or (diff_rec > max_diff)

        if not overfit_flag:
            # ✅ Found a non-overfitting threshold
            chosen_ops[name] = {
                "thr": thr,
                "precision_val": float(row["precision_val"]),
                "recall_val": float(row["recall_val"]),
                "target_recall": float(row["target_recall"]),
                "min_precision": float(row["min_precision"]),
                "f2_val": float(row["f2_val"])
            }
            thr_for_summary = thr
            note = f"✅ Selected (no overfit, diff≤{max_diff})"
            found_valid = True
            break

    if not found_valid:
        # ❌ No valid threshold found for this model
        thr_for_summary = None
        note = f"⚠️ All thresholds overfit (val/test diff > {max_diff})"
        chosen_ops[name] = {
            "thr": None,
            "precision_val": None,
            "recall_val": None,
            "target_recall": None,
            "min_precision": None,
            "f2_val": None
        }

    # For summary reporting
    if found_valid:
        best_row = row
        prec_test_final = prec_test
        rec_test_final = rec_test
        diff_prec_final = diff_prec
        diff_rec_final = diff_rec
        overfit_flag_final = False
    else:
        best_row = df_scan.iloc[0]  # fallback for reporting
        y_pred_te = (d["proba_te"] >= float(best_row["threshold"])).astype(int)
        tp = int(((y_te == 1) & (y_pred_te == 1)).sum())
        fp = int(((y_te == 0) & (y_pred_te == 1)).sum())
        prec_test_final = tp / max(tp + fp, 1)
        rec_test_final  = tp / max(int(y_te.sum()), 1)
        diff_prec_final = abs(float(best_row["precision_val"]) - prec_test_final)
        diff_rec_final = abs(float(best_row["recall_val"]) - rec_test_final)
        overfit_flag_final = True

    summary_rows.append({
        "model": name,
        "thr": thr_for_summary,
        "policy_val": f"rec≥{float(best_row['target_recall']):.2f} & prec≥{float(best_row['min_precision']):.2f}",
        "precision_val@thr": float(best_row["precision_val"]),
        "recall_val@thr": float(best_row["recall_val"]),
        "f2_val": float(best_row["f2_val"]),
        "pr_auc_test": average_precision_score(y_te, d["proba_te"]),
        "roc_auc_test": roc_auc_score(y_te, d["proba_te"]),
        "precision_test@thr": prec_test_final,
        "recall_test@thr": rec_test_final,
        "alerts_test": int((d["proba_te"] >= (thr_for_summary or 0)).sum()),
        "positives_test": int(y_te.sum()),
        "overfitting": overfit_flag_final,
        "diff_prec": diff_prec_final,
        "diff_rec": diff_rec_final,
        "note": note
    })

In [ ]:
# === Build and filter summary ===
results_df = pd.DataFrame(summary_rows).sort_values(
    ["pr_auc_test", "recall_test@thr", "precision_test@thr"],
    ascending=[False, False, False]
).reset_index(drop=True)

In [ ]:
results_df

In [ ]:
# Keep only non-overfitting (valid) models
valid_results_df = results_df[results_df["overfitting"] == False].copy().sort_values(
    ["pr_auc_test", "recall_test@thr", "precision_test@thr"],
    ascending=[False, False, False]
).reset_index(drop=True)

if valid_results_df.empty:
    print("No valid models found — all thresholds overfit.")
    best_model = None
else:
    best_model = valid_results_df.iloc[0].to_dict()
    print(f"✅ Best valid model: {best_model['model']} (thr={best_model['thr']})")

best_name = valid_results_df.iloc[0]["model"]

In [ ]:
valid_results_df[['model', 'thr', 'recall_test@thr', 'pr_auc_test', 'precision_test@thr']]

In [ ]:
best_thr = chosen_ops[best_name]["thr"]
best_model, Xv_best, Xt_best = models[best_name]

y_proba_best_te = probas[best_name]["proba_te"]
y_pred_best = (y_proba_best_te >= best_thr).astype(int)

# Compute confusion matrix
cm = confusion_matrix(y_te, y_pred_best)
tn, fp, fn, tp = cm.ravel()
labels = np.array([["TN", "FP"], ["FN", "TP"]])

# Compact confusion matrix plot
fig, ax = plt.subplots(figsize=(3.2, 3))  # reduced from (5.5, 5)
im = ax.imshow(cm, interpolation='nearest', cmap="Blues")

# Label axes
ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels(["No Outage", "Outage"], fontsize=6)
ax.set_yticklabels(["No Outage", "Outage"], fontsize=6)
ax.set_xlabel("Predicted", fontsize=6)
ax.set_ylabel("True", fontsize=6)

# Compact title
ax.set_title(
    f"Confusion Matrix — {best_name}\n"
    f"thr={best_thr:.3f} | rec≥{chosen_ops[best_name]['target_recall']:.2f}, "
    f"prec≥{chosen_ops[best_name]['min_precision']:.2f}",
    fontsize=7, pad=6
)

# Annotate cells with smaller text and color contrast
for (i, j), value in np.ndenumerate(cm):
    color = "white" if value > cm.max() / 2 else "black"
    ax.text(j, i, f"{labels[i,j]}\n{value:,}",
            ha='center', va='center', fontsize=6, fontweight='bold', color=color)

# Smaller colorbar for compact layout
cbar = plt.colorbar(im, ax=ax, fraction=0.04, pad=0.02)
cbar.ax.tick_params(labelsize=5)

plt.tight_layout(pad=0.3)
plt.show()

# Compute and print summary metrics
prec = tp / max(tp + fp, 1)
rec  = tp / max(tp + fn, 1)
f1   = (2 * prec * rec) / max(prec + rec, 1e-12)

print(f"TN={tn:,}  FP={fp:,}  FN={fn:,}  TP={tp:,}")
print(f"Precision={prec:.4f}  Recall={rec:.4f}  F1={f1:.4f}")

In [ ]:
proba_te_best = best_model.predict_proba(Xt_best)[:, 1]

# Compute precision–recall curve
p, r, thr = precision_recall_curve(y_te, proba_te_best)
ap = average_precision_score(y_te, proba_te_best)

# Find the precision and recall corresponding to the chosen threshold
# (Note: thresholds returned by precision_recall_curve are for proba >= thr)
idx = np.argmin(np.abs(thr - best_thr))  # closest threshold
best_p, best_r = p[idx], r[idx]

# Compact Precision–Recall Curve
plt.figure(figsize=(3.5, 3))  # reduced from (7, 6)
plt.plot(r, p, label=f"{best_name} (AP={ap:.3f})", lw=1.2)
plt.scatter(best_r, best_p, color="red", s=30, zorder=5, label=f"Best thr={best_thr:.2f}")

# Labels and title (smaller fonts)
plt.xlabel("Recall", fontsize=7)
plt.ylabel("Precision", fontsize=7)
plt.title("Precision–Recall Curve (Test)", fontsize=8, pad=4)

# Smaller tick labels
plt.tick_params(axis='both', which='major', labelsize=6)
plt.tick_params(axis='both', which='minor', labelsize=5)

# Legend and grid
plt.legend(fontsize=6, loc="upper right", frameon=False)
plt.grid(True, linestyle='--', alpha=0.5, linewidth=0.6)

plt.tight_layout(pad=0.3)
plt.show()

In [ ]:
if best_name == "Logistic":
    # Xt_best is a scaled array; wrap with feature_cols_all names
    Xt_best_df = pd.DataFrame(Xt_best, columns=feature_cols_all)
else:
    # Xt_best is already a DataFrame with full feature columns
    Xt_best_df = Xt_best.copy()

X_pi = Xt_best_df.copy()
y_pi = y_te.copy()

perm = permutation_importance(
    best_model, X_pi, y_pi,
    n_repeats=3, random_state=RANDOM_STATE, n_jobs=1,
    scoring="average_precision"
)

imp = pd.DataFrame({
    "feature": X_pi.columns,
    "importance_mean": perm.importances_mean,
    "importance_std": perm.importances_std
}).sort_values("importance_mean", ascending=False)

print(f"Best model: {best_name}")
print(imp.head(15))

In [ ]:
# Select top 10 features
imp_top10 = imp.head(10)

# Horizontal bar plot
plt.figure(figsize=(5, 3))
plt.barh(imp_top10["feature"], imp_top10["importance_mean"], xerr=imp_top10["importance_std"], color='skyblue')
plt.xlabel("Permutation Importance (Mean ± Std)")
plt.ylabel("Feature")
plt.title(f"Top 10 Permutation Feature Importances - {best_name} Model")
plt.gca().invert_yaxis()  # largest importance on top
plt.tight_layout()
plt.show()

In [ ]:
registry = Registry(session=session)

# pick a small sample to avoid uploading entire dataset
sample_input = pd.DataFrame(Xv_best).head(50)

best_name = best_name.lower()

# Ensure valid dependency mapping
dep_name = "catboost" if "catboost" in best_name else best_name

model_version = registry.log_model(
    model=best_model,
    model_name=f"{best_name}_outage_predictor",
    version_name="v1",
    conda_dependencies=["scikit-learn", dep_name],
    sample_input_data=sample_input
)

print("✅ Model registered!")
print(f"Model Name: {model_version.model_name}")
print(f"Version: {model_version.version_name}")

In [ ]:
# Get the model first
model = registry.get_model(f"{best_name}_outage_predictor")

# Get the specific version from the model object
model_version = model.version("v1")

# Load the actual Python model artifact
loaded_model = model_version.load()

# Run inference
y_pred_new = loaded_model.predict(Xt_best)
y_pred_new[:5]